**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



**Set global variables**

In [2]:
TXT_FOLDER_REGION = "encode_chipseq_subset"

## Define column description
The peak file is in narrowPeak format, which is a standard six field bed with four additional fields (BED6+4 format)

In [3]:
### ENCODE narrowPeak: Narrow (or Point-Source) Peaks format
dat = tribble(
    ~Name,        ~Note,
    "Chrom",      "Name of the chromosome",
    "ChromStart", "The starting position of the feature in the chromosome",
    "ChromEnd",   "The ending position of the feature in the chromosome",
    "Name",       "Name given to a region; Use '.' if no name is assigned.",
    "Score",      "Indicates how dark the peak will be displayed in the browser (0-1000).",
    "Strand",     "+/- to denote strand or orientation. Use '.' if no orientation is assigned.",
    "SignalValue","Measurement of overall (usually, average) enrichment for the region.",
    "PValue",     "Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.",
    "QValue",     "Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.",
    "Peak",       "Point-source called for this peak; 0-based offset from chromStart. Use -1 if no point-source called."
    
)

### assign and show
dat_cname = dat
fun_display_table(dat)

Name,Note
Chrom,Name of the chromosome
ChromStart,The starting position of the feature in the chromosome
ChromEnd,The ending position of the feature in the chromosome
Name,Name given to a region; Use '.' if no name is assigned.
Score,Indicates how dark the peak will be displayed in the browser (0-1000).
Strand,+/- to denote strand or orientation. Use '.' if no orientation is assigned.
SignalValue,"Measurement of overall (usually, average) enrichment for the region."
PValue,Measurement of statistical significance (-log10). Use -1 if no pValue is assigned.
QValue,Measurement of statistical significance using false discovery rate (-log10). Use -1 if no qValue is assigned.
Peak,Point-source called for this peak; 0-based offset from chromStart. Use -1 if no point-source called.


## Define file labeling

**import metadata**

In [6]:
### set directory
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname  = "metadata.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

### read table
dat = read_tsv(txt_fpath, show_col_types = FALSE)

### assign and show
dat_meta_info = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 42 12


Assay,Index_Experiment,Index_File,File_Type,Output_Type,Genome,Target,Analysis,RFA,md5sum,File_Name,File_Url
TF ChIP-seq,ENCSR121SPB,ENCFF142ZTD,bed narrowPeak,IDR thresholded peaks,GRCh38,KLF10,ENCODE4 v1.6.1 GRCh38,ENCODE4,06b317ef05d76533b65394ed228366f4,ENCFF142ZTD.bed.gz,https://www.encodeproject.org/files/ENCFF142ZTD/@@download/ENCFF142ZTD.bed.gz
TF ChIP-seq,ENCSR121SPB,ENCFF750TBY,bigWig,signal p-value,GRCh38,KLF10,ENCODE4 v1.6.1 GRCh38,ENCODE4,4886ccae5fb0343dd88faccd3c6040ee,ENCFF750TBY.bigWig,https://www.encodeproject.org/files/ENCFF750TBY/@@download/ENCFF750TBY.bigWig
Histone ChIP-seq,ENCSR000AKS,ENCFF135ZLM,bed narrowPeak,replicated peaks,GRCh38,H3K4me1,ENCODE4 v1.8.0 GRCh38,ENCODE4,b2db31175e0856a2b27c064da8e4bbb4,ENCFF135ZLM.bed.gz,https://www.encodeproject.org/files/ENCFF135ZLM/@@download/ENCFF135ZLM.bed.gz


**generate file label**

In [8]:
### get file name and generate label for each file
dat = dat_meta_info
dat = dat %>% 
    dplyr::mutate(
        FName = File_Name,
        Label = paste("encode_chipseq", Target, Index_File, sep = "_")
    ) %>%
    dplyr::select(FName, Label)

### assign and show
dat_meta_label = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 42  2


FName,Label
ENCFF142ZTD.bed.gz,encode_chipseq_KLF10_ENCFF142ZTD
ENCFF750TBY.bigWig,encode_chipseq_KLF10_ENCFF750TBY
ENCFF135ZLM.bed.gz,encode_chipseq_H3K4me1_ENCFF135ZLM


**create region label table**

In [9]:
### set directory
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder)
txt_fglob  = file.path(txt_fdiry, "*bed*")

### get file names
vec_txt_fpath = Sys.glob(txt_fglob)
vec_txt_fname = basename(vec_txt_fpath)

### init info table
dat = data.frame(
    "Folder" = txt_folder,
    "FName"  = vec_txt_fname
)

### get label
dat = dat %>% dplyr::left_join(dat_meta_label, by = "FName")

### assign and show
dat_region_label = dat
print(dim(dat))
fun_display_table(dat)

[1] 21  3


Folder,FName,Label
encode_chipseq_subset,ENCFF023IFO.bed.gz,encode_chipseq_NFE2_ENCFF023IFO
encode_chipseq_subset,ENCFF135ZLM.bed.gz,encode_chipseq_H3K4me1_ENCFF135ZLM
encode_chipseq_subset,ENCFF142ZTD.bed.gz,encode_chipseq_KLF10_ENCFF142ZTD
encode_chipseq_subset,ENCFF190CGV.bed.gz,encode_chipseq_JUN_ENCFF190CGV
encode_chipseq_subset,ENCFF242AOL.bed.gz,encode_chipseq_ZEB2_ENCFF242AOL
encode_chipseq_subset,ENCFF323WOT.bed.gz,encode_chipseq_H3K27me3_ENCFF323WOT
encode_chipseq_subset,ENCFF355MNE.bed.gz,encode_chipseq_POLR2A_ENCFF355MNE
encode_chipseq_subset,ENCFF398UQZ.bed.gz,encode_chipseq_YY1_ENCFF398UQZ
encode_chipseq_subset,ENCFF488OTN.bed.gz,encode_chipseq_KLF16_ENCFF488OTN
encode_chipseq_subset,ENCFF544LXB.bed.gz,encode_chipseq_H3K27ac_ENCFF544LXB


## Save results

In [10]:
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname  = "description.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

dir.create(txt_fdiry, showWarnings = FALSE)
dat = dat_cname
write_tsv(dat, txt_fpath)

In [11]:
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname  = "metadata.label.tsv"
txt_fpath  = file.path(txt_fdiry, txt_fname)

dir.create(txt_fdiry, showWarnings = FALSE)
dat = dat_region_label
write_tsv(dat, txt_fpath)